## Лузина Владимира РИС22-2
# Семинар 8. Использование регрессионных моделей в задачи прогнозирования

Используя имеющиеся статистические данные выполнить следующие шаги:
1) Разбить набор данных на обучающую (15 значений) и тестовую выборки (5 оставшихся
значений). Построить, используя данные из обучающей выборки, регрессионную модель
методом МНК. Провести проверку на адекватность полученной модели на тестовой выборке по
критериям Стьюдента (гипотеза о равенстве математических ожиданий) и Фишера (проверка
однородности дисперсий).
2) Построить мультипликативную модель разложения временного ряда. Для чего выделить
составляющую, полученную на первом шаге выполнения работы, а для оставшейся составляющей
выбрать вид функции на основе графика построенных значений и найти коэффициенты с
помощью МНК. Проверить качество мультипликативной модели по критериям Фишера и
Стьюдента, сравнить полученные результаты с полученными на первом шаге выполнения
задания

In [196]:
import math
import pandas as pd
import numpy as np
import scipy
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


Исходные данные взяты так, чтобы параболическая модель была самой точной

In [197]:
df = pd.DataFrame({'day': [i for i in range(1, 21)],  # дни с 1 по 20
                   'price': [2.61, 5.45, 2.88, 4.28, 2.47, 4.57, 0.15, 0.38, 1.10, 2.43, 1.23, 5.93, 1.44, 1.28, 1.43,
                             1.90, 1.96, 2.08, 3.88, 4.41],  # стоимость
                   'amount': [46, 55, 53, 43, 61, 48, 198, 151, 115, 43, 137, 89, 134, 97, 90, 133, 97., 67, 60,
                              55]})  # число продаж

#df = pd.DataFrame({'day': [i for i in range(1, 21)],  # дни с 1 по 20
#                   'price': [3.55, 4.48, 3.27, 4.32, 2.71, 4.49, 0.13, 0.5, 1.16, 3.24, 1.11, 4.98, 1.23, 1.82, 1.35,
#                             1.79, 2.01, 2.18, 3.26, 3.94],  # стоимость
#                   'amount': [50, 55, 59, 45, 60, 50, 210, 160, 140, 50, 143, 100, 130, 92, 114, 112, 97, 83, 66,
#                              60]})  # число продаж

df

,day,price,amount
0,1,2.61,46.0
1,2,5.45,55.0
2,3,2.88,53.0
3,4,4.28,43.0
4,5,2.47,61.0
5,6,4.57,48.0
6,7,0.15,198.0
7,8,0.38,151.0
8,9,1.10,115.0
9,10,2.43,43.0


Разделим данные на тест и трейн, на параметры и целевую переменную

In [198]:
X = df[["day", "price"]]
Y = df[["amount"]]
x_train, x_test, y_train, y_test = X.loc[:14, :], X.loc[15:], Y.loc[:14, :], Y.loc[15:, :]

Добавим аддитивную составляющую для трейна

In [199]:
temp = x_train["price"]
temp_d = x_train["day"]
A = x_train.drop("price", axis=1)
A = A.drop("day", axis=1)
A["A"] = [1 for i in range(len(A))]
A["day"] = temp_d
A["day_log"] = np.log2(A["day"])
A["price"] = temp
A["price_log"] = np.log2(A["price"])
A

,A,day,day_log,price,price_log
0,1,1,0.000000,2.61,1.384050
1,1,2,1.000000,5.45,2.446256
2,1,3,1.584963,2.88,1.526069
3,1,4,2.000000,4.28,2.097611
4,1,5,2.321928,2.47,1.304511
5,1,6,2.584963,4.57,2.192194
6,1,7,2.807355,0.15,-2.736966
7,1,8,3.000000,0.38,-1.395929
8,1,9,3.169925,1.10,0.137504
9,1,10,3.321928,2.43,1.280956


In [200]:
c = (np.linalg.inv(A.T @ A) @ A.T) @ y_train
c

,amount
0,65.536145
1,3.345855
2,-0.953055
3,14.355998
4,-42.108075


Окончательно запишем искомую функцию

In [201]:
def log_regression(x1: float, x2: float) -> float:
    return c.iloc[4] * math.log2(x2) + c.iloc[3] * x2 + c.iloc[2] * math.log2(x1) + c.iloc[1] * x1 + c.iloc[0]

Сделаем предикт

In [202]:
pred_train = pd.Series([round(log_regression(row["day"], row["price"])[0].item(), 0) for _, row in x_train.iterrows()])
pred_train

0      48.0
1      47.0
2      51.0
3      50.0
4      61.0
5      56.0
6     204.0
7     154.0
8     103.0
9      77.0
10    104.0
11     79.0
12    104.0
13    112.0
14    111.0
dtype: float64

In [203]:
pred_test = pd.Series([round(log_regression(row["day"], row["price"])[0].item(), 0) for _, row in x_test.iterrows()])
pred_test

0    104.0
1    106.0
2    107.0
3     98.0
4    102.0
dtype: float64

Необходимо оценить равенство тестируемых объёмов продаж и прогнозируемых значений объёмов продаж
Если они окаутся статистически равными, тогда можно говорить об удачной возможности прогнозирования, в противном случае нельзя об этом сказать
Проверять будем с помощью среднего и стандартного отклонения ()
T критерий стьюдента
s - стандартное
x - тестируемая
y = прогнозирование


Среднее значение

In [204]:
mean_test = y_test.mean().iloc[0]
mean_test

82.4

In [205]:
mean_pred = pred_test.mean()
mean_pred

103.4

отклонение

In [206]:
std_test = y_test.std().iloc[0]
std_test

32.64659247149693

In [207]:
std_pred = pred_test.std()
std_pred

3.5777087639996634

Критерий Стьюдента
Сравним математическое ожидание

In [208]:
# опытное
n = 5
t_op = (mean_pred - mean_test) / np.sqrt((std_pred / np.sqrt(n)) ** 2 + ((std_test / np.sqrt(n)) ** 2))
t_op

1.4297960367976275

In [209]:
# Критическое
t_kr = scipy.stats.t.ppf(q=1 - 0.05, df=(2 * n - 2))
t_kr

1.8595480375228421


Различия не значимы
так как $F(op) < F(cr)$

Критерий фишера
Сравним стандартное отклонение

In [210]:
n = 5
k = n - 1
f_op = std_test * std_test / std_pred / std_pred
f_op

83.265625

In [211]:
# критическое
a = 0.05
m = 1
n = 5
critical = scipy.stats.f.ppf(q=1 - a, dfn=n, dfd=n - m - 1)
critical

9.013455167522581

Различия значимы
так как $F(op) > F(cr)$

Значения среднего однородно, а значение диспирсии неоднородно, следовательно, прогнозная модель получилась не совсем удачной
Чтобы это исправить попробуем воспользоваться мультипликационной моделью

### Мультипликативная модель 

остаточное значение

In [212]:
residual_train = y_train["amount"] / pred_train
residual_train

0     0.958333
1     1.170213
2     1.039216
3     0.860000
4     1.000000
5     0.857143
6     0.970588
7     0.980519
8     1.116505
9     0.558442
10    1.317308
11    1.126582
12    1.288462
13    0.866071
14    0.810811
dtype: float64

In [213]:
residual_test = y_test.reset_index()["amount"] / pred_test
residual_test

0    1.278846
1    0.915094
2    0.626168
3    0.612245
4    0.539216
dtype: float64

cos(день)

In [214]:
cos_train_day = np.cos(x_train["day"])
cos_test_day = np.cos(x_test["day"])
cos_train_day

0     0.540302
1    -0.416147
2    -0.989992
3    -0.653644
4     0.283662
5     0.960170
6     0.753902
7    -0.145500
8    -0.911130
9    -0.839072
10    0.004426
11    0.843854
12    0.907447
13    0.136737
14   -0.759688
Name: day, dtype: float64

cos(цена)

In [215]:
cos_train_price = np.cos(x_train["price"])
cos_test_price = np.cos(x_test["price"])
cos_train_price

0    -0.862001
1     0.672522
2    -0.965979
3    -0.419041
4    -0.782832
5    -0.141908
6     0.988771
7     0.928665
8     0.453596
9    -0.757323
10    0.334238
11    0.938276
12    0.130424
13    0.286715
14    0.140332
Name: price, dtype: float64

In [216]:
A = pd.DataFrame({"A": [1 for i in range(len(x_train))], "cos(d)": cos_train_day, "cos(price)": cos_train_price})
A

,A,cos(d),cos(price)
0,1,0.540302,-0.862001
1,1,-0.416147,0.672522
2,1,-0.989992,-0.965979
3,1,-0.653644,-0.419041
4,1,0.283662,-0.782832
5,1,0.960170,-0.141908
6,1,0.753902,0.988771
7,1,-0.145500,0.928665
8,1,-0.911130,0.453596
9,1,-0.839072,-0.757323


In [217]:
c = (np.linalg.inv(A.T @ A) @ A.T) @ residual_train
c

0    0.989653
1    0.054175
2    0.096157
dtype: float64

Окончательно запишем искомую функцию

In [218]:
def residual_regression(x1: float, x2: float) -> float:
    return c.iloc[2] * np.cos(x2) + c.iloc[1] * np.cos(x1) + c.iloc[0]

Сделаем предикт

In [219]:
pred_train_residual = pd.Series([residual_regression(row["day"], row["price"]).item() for _, row in x_train.iterrows()])
pred_train_residual

0     0.936036
1     1.031777
2     0.843134
3     0.913948
4     0.929745
5     1.028025
6     1.125574
7     1.071069
8     0.983909
9     0.871374
10    1.022032
11    1.125591
12    1.051355
13    1.024631
14    0.961991
dtype: float64

In [220]:
pred_test_residual = pd.Series([residual_regression(row["day"], row["price"]).item() for _, row in x_test.iterrows()])
pred_test_residual

0    0.906685
1    0.938259
2    0.978551
3    0.972104
4    0.983125
dtype: float64

От остаточного значения вернёмся к реальному предикту

In [221]:
pred_train_2 = round(pred_train * pred_train_residual, 0)
pred_train_2

0      45.0
1      48.0
2      43.0
3      46.0
4      57.0
5      58.0
6     230.0
7     165.0
8     101.0
9      67.0
10    106.0
11     89.0
12    109.0
13    115.0
14    107.0
dtype: float64

In [222]:
pred_test_2 = round(pred_test * pred_test_residual, 0)
pred_test_2

0     94.0
1     99.0
2    105.0
3     95.0
4    100.0
dtype: float64

Среднее значение

In [223]:
mean_test = y_test.mean().iloc[0]
mean_test

82.4

In [224]:
mean_pred_2 = pred_test_2.mean()
mean_pred_2

98.6

отклонение

In [225]:
std_test = y_test.std().iloc[0]
std_test

32.64659247149693

In [226]:
std_pred_2 = pred_test_2.std()
std_pred_2

4.39317652729776

Критерий Стьюдента
Сравним математическое ожидание

In [227]:
# опытное
n = 5
t_op = (mean_pred_2 - mean_test) / np.sqrt((std_pred_2 / np.sqrt(n)) ** 2 + ((std_test / np.sqrt(n)) ** 2))
t_op

1.0996769827584025

In [228]:
# Критическое
n = 5
t_kr = scipy.stats.t.ppf(q=1 - 0.05, df=(2 * n - 2))
t_kr

1.8595480375228421

Различия не значимы
так как $F(op) < F(cr)$

Критерий фишера
Сравним стандартное отклонение

In [229]:
n = 5
k = n - 1
f_op = std_test * std_test / std_pred_2 / std_pred_2
f_op

55.22279792746113

In [230]:
# критическое
a = 0.05
m = 1
n = 5
critical = scipy.stats.f.ppf(q=1 - a, dfn=n, dfd=n - m - 1)
critical

9.013455167522581

Различия значимы
так как $F(op) < F(cr)$

Значения среднего однородно, а значение диспирсии неоднородно, следовательно, прогнозная модель получилась не совсем удачной
мультипликационная модель не помогла исправить данную проблему, но значение опытных критериев уменьшилось